# 计算思维实训
24123205 李林

SHUCES CS


# 环境配置
本次使用uv环境。请检查您的环境。

In [ ]:
!uv sync

In [ ]:
import sys
print(sys.executable)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import os
from scipy import stats


##  Networkx初次尝试
This part is  following the [official tutorial](https://networkx.org/documentation/stable/tutorial.html).[1]

In [ ]:
# Create a graph
G0 = nx.Graph()
G0.add_node(1)
G0.add_nodes_from([(4, {"color": "red"}), (5, {"color": "green"})])
print(G0)
# importing
H0 = nx.path_graph(10)
G0.add_nodes_from(H0)
print(G0)
del G0,H0

In [ ]:
# Another official example
G = nx.Graph()
G.add_edges_from([(1, 2), (1, 3)])
G.add_node(1)
G.add_edge(1, 2)
G.add_node("spam")        # adds node "spam"
G.add_nodes_from("spam")  # adds 4 nodes: 's', 'p', 'a', 'm'
G.add_edge(3, 'm')
print(G.number_of_nodes())
# manuall check the network
print(list(G.nodes))
print(list(G.edges))
print(list(G.adj[1]) ) # or list(G.neighbors(1))
print(G.degree[1])  # the number of edges incident to 1
# removing
G.remove_node(2)
G.remove_nodes_from("spam")
print(list(G.nodes))
del G

In [ ]:
# File saving & reading
G = nx.erdos_renyi_graph(50, 0.15)
filename = "./G.edgelist"
nx.write_edgelist(G, filename,data=False )
mygraph = nx.read_edgelist(filename ,nodetype=int)
print(list(mygraph.edges))
os.remove(filename)
del G,mygraph

In [ ]:
# Drawing
G = nx.erdos_renyi_graph(50, 0.1)
nx.draw(G, with_labels=False, font_weight='normal')


# 生成10万个节点的无尺度网络
## 无尺度网络的定义
> 在网络理论中，无尺度网络（Scale-free network，或称无标度网络）是带有一类特性的复杂网络，其典型特征是在网络中的大部分节点只和很少节点连接，而有极少的节点与非常多的节点连接。这种关键的节点（称为“枢纽”或“集散节点”）的存在使得无尺度网络对意外故障有强大的承受能力，但面对协同性攻击时则显得脆弱。现实中的许多网络都带有无尺度的特性，例如因特网、金融系统网络、社会人际网络等等[2]。
>
## 无标度网络的生成
无标度网络的生成主要Price模型和衍生的Barabási-Albert 模型。[3]
1. Price模型

Price模型是从研究以科学论文的引文网络中产生的。受到Herbert Simon的论文[4]的影响,Price命名为cumulative advantage。由于是论文引用模型，所以是单向图。

2. Barabási-Albert 模型

因为price模型中涉及到很多单向图的概念，为了简化讨论，这里描述Barabási-Albert模型。二者独立发现了这个模型。[5]
对于节点$i$连接到其他节点的概率$k_I$:
$$
\Pi(k_i)=\frac{k_i}{\sum_J k_J}
$$
而对于度而言，如果看作连续的过程，有
$$
\dfrac{dk_i}{dt} = \dfrac{k_i}{2t}
$$
笔者会从简单的代码实现开始，然后逐渐优化。


In [ ]:
def manualG(n=1000, m=3, seed=114514):
    
    random.seed(seed)
    G = nx.complete_graph(m)
    degrees = [G.degree(i) for i in range(m)]
    total_steps = n - m
    print_interval = max(1, total_steps // 10000)
    for new_node in range(m, n):
        targets = set()
        while len(targets) < m:
            chosen = random.choices(range(new_node), weights=degrees, k=1)[0]
            targets.add(chosen) 
        G.add_node(new_node)
        for t in targets:
            G.add_edge(new_node, t)
            degrees[t] += 1  
        degrees.append(m)
        current_step = new_node - m + 1
        if current_step % print_interval == 0 or current_step == total_steps:
            percent = 100 * current_step / total_steps
            print(f"\rProcess: {percent:6.2f}%", end='', flush=True)
    print()  
    return G
N = 1e4
G = manualG(int(N), 5, 114514)

可以看到这样的代码效率很低。尝试使用numpy进行向量化优化

In [ ]:
def manualGuseNP(n=1000, m=3, seed=114514):
    import numpy as np
    import networkx as nx
    rng = np.random.default_rng(seed)
    G = nx.complete_graph(m)
    degrees = np.array([G.degree(i) for i in range(m)], dtype=np.float64)
    total_steps = n - m
    print_interval = max(1, total_steps // 10000)
    for new_node in range(m, n):
        targets = set()
        while len(targets) < m:
            prob = degrees[:new_node] / degrees[:new_node].sum()
            chosen = rng.choice(new_node, p=prob)
            targets.add(chosen)
        G.add_node(new_node)
        for t in targets:
            G.add_edge(new_node, t)
            degrees[t] += 1
        degrees = np.append(degrees, m)
        current_step = new_node - m + 1
        if current_step % print_interval == 0 or current_step == total_steps:
            percent = 100 * current_step / total_steps
            print(f"\rProcess: {percent:6.2f}%", end='', flush=True)
    print()
    return G

N = 1e4
G = manualGuseNP(int(N), 5, 114514)


对比官方的实现:

In [ ]:
G = nx.barabasi_albert_graph(1e4, 5, seed=114514)

## 对比官方的代码进行优化
1. 计算权重
   ```
   prob = degrees[:new_node] / degrees[:new_node].sum()
   chosen = rng.choice(new_node, p=prob)
   ```
   每次都要遍历当前所有节点，但查看官方的release中
   ```
   repeated_nodes = [v for v, d in G.degree() for _ in range(d)]
   targets = random.sample(repeated_nodes, m)
   ```
   构造一个列表，其中每个节点v 重复出现 deg(v) 次。此时，从该列表中均匀随机采样一个元素，等价于按度数进行偏好依附。

2. 增量维护
   ```
   degrees[t] += 1
   degrees.append(m)
   ```
   每次更新后需重新计算整个概率分布，但官方实现中
   ```
   repeated_nodes.extend(targets)
   repeated_nodes.extend([source] * m)
   ```
   仅通过简单扩展列表维护权重信息，避免全量重算。

3. 采样去重
   ```
   targets = set()
   while len(targets) < m:
       chosen = rng.choice(new_node, p=prob)
       targets.add(chosen)
   ```
   循环可能导致多次重复采样，而官方实现中
   ```
   targets = random.sample(repeated_nodes, m)
   ```
   一次调用直接获得m个无重复目标节点。

4. 时间复杂度
   ```
   total_steps = n - m
   for new_node in range(m, n):
   ```
   每步O(n)导致整体O(n²)复杂度，但官方实现
   ```
   while source < n:
   ```
   每步O(m)使整体复杂度降至O(nm)，当m为常数时接近线性。

In [ ]:
def manualGuseNP(n=1000, m=3, seed=114514):
    rng = np.random.default_rng(seed)
    G = nx.complete_graph(m)
    repeated_nodes = []
    for i in range(m):
        repeated_nodes.extend([i] * G.degree(i))
    total_steps = n - m
    print_interval = max(1, total_steps // 10000)
    for new_node in range(m, n):
        targets = set()
        while len(targets) < m:
            idx = rng.integers(0, len(repeated_nodes))
            chosen = repeated_nodes[idx]
            targets.add(chosen)
        G.add_node(new_node)
        for t in targets:
            G.add_edge(new_node, t)
            repeated_nodes.append(t)
        repeated_nodes.extend([new_node] * m)
        current_step = new_node - m + 1
        if current_step % print_interval == 0 or current_step == total_steps:
            percent = 100 * current_step / total_steps
            print(f"\rProcess: {percent:6.2f}%", end='', flush=True)
    print()
    return G

N = 1e4
G = manualGuseNP(int(N), 5, 114514)

## Barabási-Albert 模型的扩展
Barabási-Albert 模型中m是固定的，但是这样不符合常理。NetworkX官方提供了`extended_barabasi_albert_graph`的实现。


In [ ]:
N=100
G = nx.extended_barabasi_albert_graph(N,3,0.3,0.3)
nx.draw(G, with_labels=False, font_weight='normal')

### 生成并保存网络

In [ ]:
Ntest=int(1000)
Gtest = nx.extended_barabasi_albert_graph(Ntest,3,0.2,0.1)
filename = "./Gtest.edgelist"
nx.write_edgelist(Gtest, filename, data=False)


In [ ]:
N=int(1e4)
G = nx.extended_barabasi_albert_graph(N, 5, 0.1,0.1)

但是使用extended的BA算法，发现速度过慢到难以接受的程度。继续通过IDE的快速查看实现功能进行跳转，检查NetworkX的实现：
```python
#……
eligible_nodes = [nd for nd, deg in G.degree() if deg < clique_degree]
#……
nbr_nodes = list(G[node])
#……
dest_node = seed.choice([nd for nd in attachment_preference if nd not in prohibited_nodes])
#……
attachment_preference.remove(src_node)
```
在每个节点，重新计算了`eligible_nodes`。在这些操作中，由于动态的变化导致不能一次性生成一个持续可用的节点相邻列表。

### 运用自动机理论进行解释


1. 经典 BA 模型

在经典 BA 模型中，系统的状态可以用度数序列 $\mathbf{d}(t) = (d_1(t), d_2(t), \dots, d_{n(t)}(t))$ 完全描述。

转移概率仅依赖当前状态：
$$
P(\mathbf{d}(t+1) | \mathbf{d}(t), \mathbf{d}(t-1), \dots) = P(\mathbf{d}(t+1) | \mathbf{d}(t))
$$

具体地，新节点 $v_{t+1}$ 连接到节点 $i$ 的概率为：
$$
\Pi(i) = \frac{d_i(t)}{\sum_{j=1}^{t} d_j(t)} = \frac{d_i(t)}{2mt}
$$



2. 扩展 BA 模型的非马尔可夫性


扩展 BA 引入了两种额外操作，破坏了马尔可夫性：

边重连操作 (概率 $q$)

当重连边 $(u,v)$ 时，选择新目标 $w$ 的概率为：
$$
\Pi(w | u, v, \mathcal{G}t) = \begin{cases}
\frac{d_w(t)}{\sum{k \notin N(u) \cup {u}} d_k(t)} & \text{if } w \notin N(u) \cup {u} \
0 & \text{otherwise}
\end{cases}
$$

新增边操作 (概率 $p$)

选择源节点 $u$ 和目标节点 $v$ 的联合概率：
$$
P(u,v | \mathcal{G}_t) = P(u | \mathcal{G}_t) \cdot P(v | u, \mathcal{G}_t)
$$
其中：
$$
P(u | \mathcal{G}_t) \propto \mathbb{I}[d_u(t) < t-1] \quad \text{(度数未饱和)}
$$
$$
P(v | u, \mathcal{G}_t) \propto d_v(t) \cdot \mathbb{I}[v \notin N(u) \cup {u}]
$$

3. 状态空间的维度

经典 BA：状态空间维度 = $n(t)$（度数序列）

扩展 BA：状态空间需要包含：


度数序列 $\mathbf{d}(t)$

邻接矩阵 $\mathbf{A}(t) = [a_{ij}(t)]$，其中 $a_{ij}(t) = \mathbb{I}[(i,j) \in E(t)]$


因此状态空间维度从 $O(n)$ 膨胀到 $O(n^2)$。

4. 数学复杂度对比


经典 BA

状态转移：$O(m)$ 时间

状态表示：$O(t)$ 空间（repeated_nodes 数组）

总复杂度：$O(nm)$


扩展 BA：

状态转移：$O(n + m)$ 时间（需要重新计算 eligibility）

状态表示：$O(n^2)$ 隐式空间（完整图结构）

总复杂度：$O(n^2m)$


不妨碍网络的性质，仍然使用传统的模型

In [ ]:
N=int(1e6)
G = nx.barabasi_albert_graph(1e6, 5, seed=114514)
filename = "./G.edgelist"
nx.write_edgelist(G, filename, data=False)

# 统计度分布图


In [ ]:
degree_hist_test = nx.degree_histogram(Gtest)
print(degree_hist_test)

In [ ]:
degree_hist = nx.degree_histogram(G)

查看nx的实现
```
counts = Counter(d for n, d in G.degree())
    return [counts.get(i, 0) for i in range(max(counts) + 1 if counts else 0)]
```
Counter是Python原生的组件，通过缓存机制高校计数。解释型语言高效运行必须熟悉语言本身。

In [ ]:

def drawPowerLaw(degree_hist, min_freq=1, outlier_threshold=2.0):
    k = np.arange(len(degree_hist))
    freq = np.array(degree_hist)
    mask = (freq >= min_freq) & (k > 0)
    log_k = np.log(k[mask])
    log_freq = np.log(freq[mask])
    slope_init, intercept_init, r, p, se = stats.linregress(log_k, log_freq)

    fitted_values = intercept_init + slope_init * log_k
    residuals = log_freq - fitted_values
    residual_std = np.std(residuals)

    inlier_mask = np.abs(residuals) <= outlier_threshold * residual_std

    log_k_clean = log_k[inlier_mask]
    log_freq_clean = log_freq[inlier_mask]

    slope, intercept = stats.siegelslopes(log_freq_clean, log_k_clean)
    r2 = None

    plt.figure(figsize=(10, 6))
    plt.scatter(log_k, log_freq, alpha=0.5, label='All points', s=10)
    plt.scatter(log_k_clean, log_freq_clean, alpha=0.7, label='Filtered inliers', s=20)
    plt.plot(log_k_clean, intercept + slope * log_k_clean, 'r-', 
             label=f'Fit: slope = {slope:.3f}' + (f', R² = {r2:.3f}' if r2 else ''))
    plt.xlabel('log(Degree k)')
    plt.ylabel('log(Frequency)')
    plt.title('Log-Log Degree Distribution with Outlier Filtering')
    plt.legend()
    plt.grid(True, ls='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

drawPowerLaw(degree_hist_test, min_freq=3, outlier_threshold=2.0)


In [ ]:
drawPowerLaw(degree_hist, min_freq=3, outlier_threshold=2.0)


# 平均最短路径

In [ ]:
avgShortestPath = nx.average_shortest_path_length(Gtest)
print(f"{avgShortestPath:.3f}")

但是当处理1e5 这种数量级，会效率很低。尝试使用MonteCarlo方法

In [ ]:
def avgShortestPathMonteCarlo(G,  tolerance=0.01, max_samples=100000):
    nodes = list(G.nodes())
    distances = []
    prev_mean = float('inf')
    
    for i in range(max_samples):
        source = random.choice(nodes)
        target = random.choice(nodes)
        while target == source:
            target = random.choice(nodes)
        
        dist = nx.shortest_path_length(G, source=source, target=target)
        distances.append(dist)
        
        if len(distances) > 100:
            current_mean = sum(distances) / len(distances)
            if abs(current_mean - prev_mean) < tolerance:
                break
            prev_mean = current_mean
    
    return sum(distances) / len(distances), len(distances)

TestavgShortestPath, Testsamples = avgShortestPathMonteCarlo(Gtest)
print(f"average shortest path length: {TestavgShortestPath:.3f}")
print(f"samples: {Testsamples}")

In [ ]:
ShortestPath, samples = avgShortestPathMonteCarlo(G)
print(f"average shortest path length: {avgShortestPath:.3f}")
print(f"samples: {samples}")

# 聚类系数

In [ ]:
def avgClusteringMonteCarlo(G, tolerance=10e-6, max_samples=100000):
    nodes = list(G.nodes())
    clustering_coeffs = []
    prev_mean = float('inf')
    
    for i in range(max_samples):
        node = random.choice(nodes)
        clustering = nx.clustering(G, node)
        clustering_coeffs.append(clustering)
        
        if len(clustering_coeffs) > 100000:
            current_mean = sum(clustering_coeffs) / len(clustering_coeffs)
            if abs(current_mean - prev_mean) < tolerance:
                break
            prev_mean = current_mean
    
    return sum(clustering_coeffs) / len(clustering_coeffs), len(clustering_coeffs)
# G = nx.barabasi_albert_graph(1e4, 5, seed=114514)
# Gtest = nx.barabasi_albert_graph(1e4, 5, seed=114514)
TestavgClustering, Testclustsamples = avgClusteringMonteCarlo(Gtest)
print(f"clustering coefficient: {TestavgClustering:.8f}")
print(f"samples: {Testclustsamples}")

In [ ]:
avgClustering, clustsamples = avgClusteringMonteCarlo(G)
print(f"clustering coefficient: {avgClustering:.8f}")
print(f"samples: {clustsamples}")

# 总结
本次计算思维实训通过NetworkX实现了复杂网络建模与分析。Python的简洁语法和NetworkX的直观接口便于快速实现理论模型，但在处理十万级节点时，纯Python实现的性能瓶颈明显。手动实现偏好依附机制时，原始循环结构导致O(n²)时间复杂度，效率低下。

优化过程包含三方面：一是算法层面，参考NetworkX官方实现的"重复节点列表"技巧，通过空间换时间将采样复杂度降至常数级；二是利用NumPy等C/C++后端库进行向量化操作；三是针对全局指标计算采用蒙特卡洛抽样，以精度换时间。同时，分析nx.degree_histogram发现Counter的哈希与缓存机制对性能至关重要。

实训中还发现NetworkX支持后端调度机制，可通过第三方加速库(如GPU后端)进一步提升性能；图生成器的多样性也为后续研究提供了更多选择。本次实训不仅加深了对无尺度网络特性的理解，更强化了大数据场景下算法设计与性能优化的实践能力，认识到Python在科学计算中需结合向量化、算法优化和近似计算策略才能有效处理大规模数据。

---
# 参考文献
- [1]NETWORKX. Tutorial — NetworkX 3.4 documentation [EB/OL]. (2024) [2024-06-15]
- [2]张皓. 《灌园先生日记》中家族社会网络之分析研究（1927—1932）[D]. 彰化: 国立彰化师范大学历史学研究所, 2024. 
- [3]NEWMAN M E J. 网络科学引论[M]. 郭世泽, 陈哲, 译. 北京: 电子工业出版社, 2014.
- [4]Simon, H. A., On a class of skew distribution functions, Biometrika 42, 425–440 (1955)
- [5]ALBERT R, BARABÁSI A L. Statistical mechanics of complex networks[J]. Reviews of Modern Physics, 2002, 74(1): 47–97. DOI:10.1103/RevModPhys.74.47.
- [6]李林. 利用Python对于上海大学计算机相关的课程与教师的网络初步分析[R]. 上海: 上海大学, 2024.